# rosalie dev

Notebook purpose:

- Develop `rosalie` package and templates for working with it

In [3]:
import logging
import os

import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.weightstats import ttest_ind

import rosalie as ro

# Silence info logging of root logger to silence Cuped logging
logging.getLogger().setLevel(logging.WARNING)

# pd.set_option('display.max_rows', 500)

%config InlineBackend.figure_format ='retina'
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

## Customer data

In [4]:
fp = '/Users/fabian.gunzinger/tmp/rosalie_dev.parquet'

if os.path.exists(fp):
    df = pd.read_parquet(fp)
else:
    df = ro.read_sample_data()
    df.to_parquet(fp)
print(df.shape)

df.head(3)

(100000, 15)


,id,post_timeframe,good_order_subject_conversion,order_daily_conversion,order_subject_conversion,contact_click_subject_conversion,contact_ticket_proportion_of_good_orders,avg_food_price,pre_timeframe,good_order_subject_conversion_pre,order_daily_conversion_pre,order_subject_conversion_pre,contact_click_subject_conversion_pre,contact_ticket_proportion_of_good_orders_pre,avg_food_price_pre
0,JE:UK:9530705,timeframe >= '2023-07-01' AND timeframe <= '20...,0.0,0.0,0.0,NaN,NaN,NaN,timeframe >= '2023-05-01' AND timeframe <= '20...,0.0,0.0,0.0,NaN,NaN,NaN
1,e4d2457e-1c68-410d-8088-0a09976f6809,timeframe >= '2023-07-01' AND timeframe <= '20...,0.0,0.0,0.0,NaN,NaN,NaN,timeframe >= '2023-05-01' AND timeframe <= '20...,0.0,0.0,0.0,NaN,NaN,NaN
2,03445EFA-7A76-4062-BF38-131F25A6D6AD,timeframe >= '2023-07-01' AND timeframe <= '20...,0.0,0.0,0.0,NaN,NaN,NaN,timeframe >= '2023-05-01' AND timeframe <= '20...,0.0,0.0,0.0,NaN,NaN,NaN


In [5]:
METRICS = [
    # 'good_order_subject_conversion',
    # 'order_daily_conversion',
    # 'order_subject_conversion',
    # 'contact_ticket_proportion_of_good_orders',
    'avg_food_price'
    ]

In [9]:
def welch_t_test(df, metric):
    """Return p-value of Welch's t-test."""
    control_sample = df[df["assignments"] == "control"][metric]
    variant_sample = df[df["assignments"] == "treatment"][metric]
    _, p, _ = ttest_ind(control_sample, variant_sample, usevar="unequal")
    return p


def traditional_cuped(df, metric):
    """Run traditional CUPED and return p-value."""
    def _cuped_adjusted_metric(df, metric, metric_pre):
        dd = df.dropna(subset=[metric, metric_pre])
        m = np.cov([dd[metric], dd[metric_pre]])
        theta = m[0, 1] / m[1, 1]
        y = df[metric]
        x = df[metric_pre]
        return (y - x * theta).fillna(y)

    df = df.copy()
    df[metric] = _cuped_adjusted_metric(df, metric, f"{metric}_pre")
    control_sample = df[df["assignments"] == "control"][metric]
    variant_sample = df[df["assignments"] == "treatment"][metric]
    _, p, _ = ttest_ind(control_sample, variant_sample, usevar="unequal")
    return p

In [12]:
evaluators = [welch_t_test, traditional_cuped]

results = []
for metric in METRICS:

    cols = ['id', metric, f'{metric}_pre']
    data = df[cols].dropna(subset=[metric])

    eval = ro.Simulator(
        df=data,
        metrics=[metric],
        evaluators=evaluators,
        sample_min=1000,
        sample_max=30_000,
        num_steps=10,
        mdes=[0.01],
        num_runs=10,
        # sample_timestamps=False,
        # alpha=0.05,
        # random_seed=2312,
        verbose=False,
    )
    result = eval.run()
    results.append(result)
    # print(result.data.head())
    result.plot().display()

31698 31698


100%|██████████| 10/10 [00:01<00:00,  7.72it/s]


alt.VConcatChart(...)